In [2]:
from minio import Minio
import pandas as pd
import os
import io
from datetime import datetime, timedelta


In [3]:
client = Minio(endpoint="localhost:9000", access_key='a5926TSNVC2r9J4Y2Eqh', secret_key='3YBQqcerjz5TsV8X851gi3Rl7YNclYQ6UD1MrEPY', secure=False)

In [4]:
#DEFINITIONS
tbl_list = ['users', 'products', 'transactions', 'transaction_detail']
bucket_name = 'snapshot'

#DEFINE CURRENT TIME VARIABLES
current_dt_string = datetime.now()
current_date_full = current_dt_string.strftime("%d%m%Y%H%M%S")
current_year = current_dt_string.strftime("%Y")
current_month = current_dt_string.strftime("%m")
current_date = current_dt_string.strftime("%d")

#DEFINE PREVIOUS DAY TIME VARIABLES

prev_dt_string = current_dt_string - timedelta(days=1)
prev_full_date = prev_dt_string.strftime("%d%m%Y%H%M%S")
prev_year = prev_dt_string.strftime("%Y") # prev_year means that the year that last day's data was on, not literally 'last year'
prev_month = prev_dt_string.strftime("%m") # the same rule applies for prev_month and prev_date.
prev_date = prev_dt_string.strftime("%d")


In [5]:

def create_df_for_t1_day():
    df_t1 = []
    for table in tbl_list:
        # t1_dt_obj_list = client.list_objects(bucket_name=bucket_name, prefix=f'{bucket_name}/{table}/2024/06/08/')
        t1_dt_obj_list = client.list_objects(bucket_name=bucket_name, prefix=f'{bucket_name}/{table}/{prev_year}/{prev_month}/{prev_date}/')
        for obj in t1_dt_obj_list:            
            file_path = os.path.basename(obj.object_name)
            object_key = obj.object_name
            print(f'object_key: {object_key}, file_path: {file_path}')

            try:
                response_t1 = client.get_object(bucket_name,object_name=object_key)
                buffer = io.BytesIO(response_t1.read())
                df = pd.read_parquet(buffer)
                df_t1.append(df)
                print(df_t1)       
            
            finally:
                response_t1.close()
                buffer.close()
                response_t1.release_conn()
    return df_t1

In [7]:
def create_df_for_t2_day():
    for table in tbl_list:
        t2_dt_obj_list = client.list_objects(bucket_name=bucket_name, prefix=f'{bucket_name}/{table}/{prev_year}/{prev_month}/{prev_date}/')
        for obj in t2_dt_obj_list:            
            file_path = os.path.basename(obj.object_name)
            object_key = obj.object_name
            print(f'object_key: {object_key}, file_path: {file_path}')

            try:
                response_t2 = client.get_object(bucket_name, object_name=object_key)
                buffer = io.BytesIO(response_t2.read())
                df_t2 = pd.read_parquet(buffer)
                print(df_t2)       
                
            finally:
                response_t2.close()
                buffer.close()
                response_t2.release_conn()
    return df_t2

In [8]:
def create_df_for_changes():
    t1_df = create_df_for_t1_day()
    t2_df = create_df_for_t2_day()
    print(t1_df)
    print(t2_df)

In [9]:
create_df_for_changes()

object_key: snapshot/users/2024/06/11/users_snapshot_11062024070127.parquet, file_path: users_snapshot_11062024070127.parquet
[         id               name                            email  \
0    RW6065           JON YANG        jon24@adventure-works.com   
1    IZ6732       EUGENE HUANG     eugene10@adventure-works.com   
2    WB7662       RUBEN TORRES      ruben35@adventure-works.com   
3    YZ6738  ELIZABETH JOHNSON   elizabeth5@adventure-works.com   
4    MV5719         JULIO RUIZ       julio1@adventure-works.com   
..      ...                ...                              ...   
186  MX6782       John Acevedo           jennifer56@example.org   
187  SX7709        Robert Hale               leah73@example.net   
188  MY3640    Robert Griffith         jacqueline13@example.com   
189  BV3594     Elizabeth York          kellyashley@example.net   
190  QY3597    Jessica Kennedy  christophermcknight@example.net   

             created_at  
0   2024-05-15 13:30:00  
1   2024-05-15 1